In [2]:
import requests
import pandas as pd  

/var/folders/lq/0q3fzmb57s3c187k6gjyx3_00000gn/T/ipykernel_10724/4035540452.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
#MI Annual Plant Data to get Plant Codes
api = '' #Register for an API key here https://www.eia.gov/opendata/register.php

#data features to use
data = ['total-consumption','total-consumption-btu','generation','gross-generation',
        'consumption-for-eg','consumption-for-eg-btu','average-heat-content']


def api_url (data=['total-consumption'], fuel=['WH','NG'], states=[], plant=[],
             frequency='monthly', start=[], end=[], primeMover=['ALL'], api=api):
    "Given data parameters, creates an api URL for the EIA"
    
    datas = ''.join([f'data[]={x}&' for x in data])
    fuels = ''.join([f'facets[fuel2002][]={x}&' for x in fuel])
    plants = ''.join([f'facets[plantCode][]={x}&' for x in plant])
    statess = ''.join([f'facets[state][]={x}&' for x in states])
    mover = ''.join([f'facets[primeMover][]={x}&' for x in primeMover])
    freq = f'frequency={frequency}&'
    start = ''.join([f'start={x}&' for x in start])
    end = ''.join([f'end={x}&' for x in end])
    api_key = f'api_key={api}'

    url = f'https://api.eia.gov/v2/electricity/facility-fuel/data?{datas}{fuels}{statess}{mover}{plants}{freq}{start}{end}{api_key}'
    print(url)
    return url

In [4]:
def pd_from_api (data=['total-consumption'], fuel=['WH','NG'], states=[], plants=[],
             frequency='monthly', start=[], end=[], primeMover=['ALL'], api=api):
    """Given data parameters, such as state and time frame, uses api_url to pull data from EIA and create
    a pandas data frame - can be used to find plant codes for a specific state, or to pull data for a plant"""
    if plants==[]:
        url = api_url(data=data, fuel=fuel, states=states,
             frequency=frequency, start=start, end=end, primeMover=primeMover, api=api)
        r = requests.get(url)
        txt = r.json()
        df = pd.DataFrame(txt['response']['data'])
    else:
        for code in plants:
            url = api_url(data=data, fuel=fuel, states=states, plant=[code],
                 frequency=frequency, start=start, end=end, primeMover=primeMover, api=api)
            r = requests.get(url)
            txt = r.json()
            temp_df = pd.DataFrame(txt['response']['data'])
            try:
                df = pd.concat([temp_df, df], ignore_index=True, axis=0)
            except:
                df = temp_df
            
    df['plantCode'] = df['plantCode'].astype(int) #so can merge with location data Plant_Code
    
    return df


In [5]:
def add_location(df, output_name):
    """Input: df with plantCode and an output file name
    Output: df merged with location data for longitude and latitude, and saved as CSV"""
    location_df = pd.read_csv('Power_Plants_in_the_U.S.csv')
    combo_df = pd.merge(df, location_df, how='left', left_on='plantCode', right_on='Plant_Code')
    combo_df.to_csv(output_name, index=False)
    return combo_df

In [7]:
def pull_state (state, start=['2019-01-01'],end=['2023-12-31'], primeMover=['ALL']):
    """Input: 2-letter state code, start and end dates in a list format
    Output: df of total monthly natural gas consumption data for all plants in a given state for a given data range
        primeMover default is 'ALL' and date range is the last 5 years (2019-2023)"""
    plants = pd_from_api(states=[state],frequency='annual')
    df = pd_from_api(data=data, states=[state],frequency='monthly', plants=plants.plantCode.unique(), 
                     start=start, end=end)
    #df.to_csv(f'{state}_Plant.csv', index=False)
    
    #add in location data and save as 'Plant_Location_St.csv'
    df_location = add_location(df, f'Plant_Location_{state}.csv')
    return  print(f'{state} data downloaded and saved') #df_location uncomment to explore

In [12]:
continental_states = ['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
           'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [ ]:
#Run to pull data for all states and save each as a seperate csv: 'Plant_Location_STATE.csv'
for state in continental_states:
    pull_state(state)

In [9]:
sample_states = ['FL','MI', 'TX', 'WA']

In [25]:
def data_stats():
    '''Prints out stats on data size and features for EIA Consumption data pulled from API and Location data'''
    locations = pd.read_csv('Power_Plants_in_the_U.S.csv')
    print(f'{len(locations)} rows in Location Data')
    print(f'{len(locations.columns)} features in Location Data')
    print(f'Features in Location Data: {locations.columns}')
    us = 0
    sample = 0
    features = []
    for state in continental_states:
        df  = pd.read_csv(f'Plant_Location_{state}.csv')
        if us ==0:
            features = df.columns
        us += len(df)
        if state in sample_states:
            sample += len(df)
    print(f'{us} rows in US NG Consumption Data')
    print(f'{sample} rows in Sample NG Consumption Data')
    print(f'{len(features)-len(locations.columns)} features in NG Consumption Data')
    print(f'Features in NG Consumption Data: {features}')



In [26]:
data_stats()

12008 rows in Location Data
35 features in Location Data
Features in Location Data: Index(['X', 'Y', 'FID', 'OBJECTID', 'Plant_Code', 'Plant_Name', 'Utility_ID',
       'Utility_Na', 'sector_nam', 'Street_Add', 'City', 'County', 'State',
       'Zip', 'PrimSource', 'source_des', 'tech_desc', 'Install_MW',
       'Total_MW', 'Bat_MW', 'Bio_MW', 'Coal_MW', 'Geo_MW', 'Hydro_MW',
       'HydroPS_MW', 'NG_MW', 'Nuclear_MW', 'Crude_MW', 'Solar_MW', 'Wind_MW',
       'Other_MW', 'Source', 'Period', 'Longitude', 'Latitude'],
      dtype='object')
130213 rows in US NG Consumption Data
23226 rows in Sample NG Consumption Data
22 features in NG Consumption Data
Features in NG Consumption Data: Index(['period', 'plantCode', 'plantName', 'fuel2002', 'fuelTypeDescription',
       'state', 'stateDescription', 'primeMover', 'total-consumption',
       'total-consumption-units', 'total-consumption-btu', 'generation',
       'gross-generation', 'consumption-for-eg', 'consumption-for-eg-units',
       'c